## Lab 5 - Interactive Plotting

In [2]:
import pandas as pd

from bokeh.plotting import figure, show 
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from ipywidgets import interact
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

Documentation: https://docs.bokeh.org/en/0.8.2/docs/user_guide/intro.html


* figure - refers to the entire plot
* glyph -  basic visual building blocks, e.g. lines, rectangles, circles, etc.
* output_file() - specifies how to output the plot
* show() - displays the plot

### Importing & merging data

In [3]:
# Download reported crime data from 2019

def get_data_chicago(id):
    '''
    Connect to the chicago data portal API and returns a dataframe
    '''
    
    url = f'https://data.cityofchicago.org/api/views/{id}/rows.csv?accessType=DOWNLOAD'
    df = pd.read_csv(url)

    
    return df


def crime_data():
    ''' 
    Pulls 2019 and 2020 Chicago crime data, joins them together, and cleans them
    '''
    
    crimes2019 = get_data_chicago('w98m-zvie')
    crimes2020 = get_data_chicago('qzdf-xmn8')

    crimes = pd.concat([crimes2019, crimes2020])

#     crimes = crimes[['ID', 'Date', 'Primary Type', 'FBI Code', 
#             'Arrest', 'Community Area', 'Year','Location']]

    return crimes


def community_area():
    ''' 
    Pulls Chicago community area data
    '''
    #Chicago community areas dataset
    comm_areas = get_data_chicago('igwz-8jzy')

    #select only relevant columns
    comm_areas = comm_areas[['COMMUNITY', 'AREA_NUMBE']]
    return comm_areas

def merge_crime_comm(crimes, comm_areas):
    ''' 
    Merges crime and community area data
    '''
    #merge crime df and community areas df
    merged = crimes.merge(comm_areas, left_on="Community Area", 
                          right_on="AREA_NUMBE", how="inner")
    return merged

# run the functions
crimes = crime_data()
comm_areas = community_area()
merged = merge_crime_comm(crimes, comm_areas)
merged.head()

ID Case Number                    Date                  Block  IUCR  \
0  12210140    JD418883  12/01/2019 12:00:00 AM    016XX W HARRISON ST  0265   
1  11861652    JC473534  10/15/2019 11:40:00 AM    004XX W HARRISON ST  1120   
2  11832807    JC438403  09/18/2019 11:50:00 AM  002XX N MILWAUKEE AVE  041A   
3  11903511    JC499929  11/05/2019 02:33:00 PM       002XX S CANAL ST  2091   
4  12002827    JD178180  12/10/2019 01:09:00 PM       010XX S CANAL ST  0810   

              Primary Type           Description  \
0  CRIMINAL SEXUAL ASSAULT    AGGRAVATED - OTHER   
1       DECEPTIVE PRACTICE               FORGERY   
2                  BATTERY  AGGRAVATED - HANDGUN   
3                NARCOTICS      FORFEIT PROPERTY   
4                    THEFT             OVER $500   

                    Location Description  Arrest  Domestic  ...  FBI Code  \
0            HOSPITAL BUILDING / GROUNDS   False     False  ...        02   
1         GOVERNMENT BUILDING / PROPERTY    True     False  ...        10   
2                               SIDEWALK    True     False  ...       04B   
3  OTHER RAILROAD PROPERTY / TRAIN DEPOT    True     False  ...        18   
4                        OTHER (SPECIFY)    True     False  ...        06   

   X Coordinate  Y Coordinate  Year              Updated On   Latitude  \
0           NaN           NaN  2019  11/03/2020 03:50:58 PM        NaN   
1     1173461.0     1897593.0  2019  11/03/2020 03:48:30 PM  41.874416   
2     1172989.0     1901805.0  2019  10/31/2020 03:46:39 PM  41.885985   
3           NaN           NaN  2019  10/24/2020 03:49:12 PM        NaN   
4     1173291.0     1895816.0  2019  10/18/2020 03:44:15 PM  41.869544   

   Longitude                       Location       COMMUNITY  AREA_NUMBE  
0        NaN                            NaN  NEAR WEST SIDE          28  
1 -87.638592    (41.8744162, -87.638591667)  NEAR WEST SIDE          28  
2 -87.640200   (41.885984671, -87.64019973)  NEAR WEST SIDE          28  
3        NaN                            NaN  NEAR WEST SIDE          28  
4 -87.639269  (41.869543766, -87.639268581)  NEAR WEST SIDE          28  

[5 rows x 24 columns]

In [13]:
count_by_community = merged.groupby(['COMMUNITY','Primary Type']).size().to_frame('COUNT').reset_index()
count_by_community.head()

COMMUNITY                       Primary Type  COUNT
0  ALBANY PARK                              ARSON      8
1  ALBANY PARK                            ASSAULT    310
2  ALBANY PARK                            BATTERY    683
3  ALBANY PARK                           BURGLARY    176
4  ALBANY PARK  CONCEALED CARRY LICENSE VIOLATION      1

### Plotting bar graph

In [58]:
def plot_crimes(df, neighborhood):
    df = df[df['COMMUNITY']==neighborhood]
    df = df.sort_values('COUNT', ascending=False).head(10)
    
    barplot = figure(title='{} Crimes'.format(neighborhood), # set up Figure object
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'],
                  y_range=(0,600),
                  plot_width=800, 
                  plot_height=600)

    barplot.vbar(x=df['Primary Type'], top=df['COUNT'],width=0.9)
    barplot.xaxis.major_label_orientation = "vertical"
    barplot.y_range.start = 0
    
    
    show(barplot)
    
plot_crimes(count_by_community, 'BRIDGEPORT')


## Adding Interactivity with Decorators


**What is a decorator?**

Decorators can add functionality to an existing code.

In [69]:
# Example from https://www.datacamp.com/community/tutorials/decorators-python

# this is the function that "wraps" around the other function - needs to have a wrapper subfunction
def uppercase_decorator(function):
    def wrapper():
        func = function()
        make_uppercase = func.upper()
        return make_uppercase

    return wrapper

In [70]:
@uppercase_decorator # decorator
def say_hi():
    return 'hello there'

say_hi()

'HELLO THERE'

### Widgets
https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html

### Adding tabs

In [64]:
neighborhoods = merged['COMMUNITY'].unique()

def plot_crimes2(df, neighborhood):
    df = df[df['COMMUNITY']==neighborhood]
    df = df.sort_values('COUNT', ascending=False).head(10)
    
    # bar plot
    barplot = figure(title='{} Crimes'.format(neighborhood), # set up Figure object
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'])

    barplot.vbar(x=df['Primary Type'], top=df['COUNT'],width=0.9)
    barplot.xaxis.major_label_orientation = "vertical"
    bar_panel = Panel(child=barplot, title='Bar plot')
    
    # line plot
    lineplot = figure(title='{} Crimes'.format(neighborhood),
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'])

    lineplot.line(df['Primary Type'], df['COUNT'])
    lineplot.xaxis.major_label_orientation = "vertical"
    line_panel = Panel(child=lineplot, title='Line plot')
    
    # scatter plot
    scatterplot = figure(title='{} Crimes'.format(neighborhood),
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'])

    scatterplot.circle(df['Primary Type'], df['COUNT'])
    scatterplot.xaxis.major_label_orientation = "vertical"
    scatter_panel = Panel(child=scatterplot, title='Scatter plot')

    tabs = Tabs(tabs=[bar_panel, line_panel, scatter_panel])
    return tabs
    

@interact(neighborhood=neighborhoods)
def make_plot_for(neighborhood='BRIDGEPORT'):
    tabs = plot_crimes2(count_by_community, neighborhood)
    show(tabs)